In [5]:
import wordle_simulation as ws
import wordle_solver_utils as wu
import pandas as pd
import plotly.express as px

In [7]:
words = ws.load_words()
suggested_starting_words = wu.find_options(words, 10)
suggested_starting_words

['soare',
 'arose',
 'aeros',
 'serai',
 'reais',
 'raise',
 'arise',
 'aesir',
 'aloes',
 'toeas']

In [8]:
list_of_starting_words = ['irate'] + suggested_starting_words

dict_list = [ws.run_simulation(1000, word) for word in list_of_starting_words]

In [16]:
df = pd.DataFrame(dict_list, index=list_of_starting_words)
df = df.reindex(sorted(df.columns), axis=1)
df = df.fillna(0)
df

,-1,1,2,3,4,5,6
irate,121,0.0,12,165,346,245,111
soare,111,0.0,21,146,344,239,139
arose,113,1.0,13,144,357,249,123
aeros,112,0.0,16,145,336,258,133
serai,124,0.0,15,176,363,212,110
reais,126,0.0,9,155,328,258,124
raise,118,0.0,19,189,317,235,122
arise,139,0.0,6,173,357,234,91
aesir,116,0.0,9,190,324,229,132
aloes,112,1.0,19,142,334,268,124


In [17]:
px.bar(df)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'variable=-1<br>index=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': '-1',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '-1',
              'offsetgroup': '-1',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['irate', 'soare', 'arose', 'aeros', 'serai', 'reais', 'raise', 'arise',
                          'aesir', 'aloes', 'toeas'], dtype=object),
              'xaxis': 'x',
              'y': array([121., 111., 113., 112., 124., 126., 118., 139., 116., 112., 117.]),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'variable=1<br>index=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': '1',
              'marker': {'color': '#EF553B', 'pattern': {'shape': ''}},
              'name': '1',
              'offsetgroup': '1',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['irate', 'soare', 'arose', 'aeros', 'serai', 'reais', 'raise', 'arise',
                          'aesir', 'aloes', 'toeas'], dtype=object),
              'xaxis': 'x',
              'y': array([0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.]),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'variable=2<br>index=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': '2',
              'marker': {'color': '#00cc96', 'pattern': {'shape': ''}},
              'name': '2',
              'offsetgroup': '2',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['irate', 'soare', 'arose', 'aeros', 'serai', 'reais', 'raise', 'arise',
                          'aesir', 'aloes', 'toeas'], dtype=object),
              'xaxis': 'x',
              'y': array([12., 21., 13., 16., 15.,  9., 19.,  6.,  9., 19., 14.]),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'variable=3<br>index=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': '3',
              'marker': {'color': '#ab63fa', 'pattern': {'shape': ''}},
              'name': '3',
              'offsetgroup': '3',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['irate', 'soare', 'arose', 'aeros', 'serai', 'reais', 'raise', 'arise',
                          'aesir', 'aloes', 'toeas'], dtype=object),
              'xaxis': 'x',
              'y': array([165., 146., 144., 145., 176., 155., 189., 173., 190., 142., 162.]),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'variable=4<br>index=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': '4',
              'marker': {'color': '#FFA15A', 'pattern': {'shape': ''}},
              'name': '4',
              'offsetgroup': '4',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['irate', 'soare', 'arose', 'aeros', 'serai', 'reais', 'raise', 'arise',
                          'aesir', 'aloes', 'toeas'], dtype=object),
              'xaxis': 'x',
              'y': array([346., 344., 357., 336., 363., 328., 317., 357., 324., 334., 334.]),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'variable=5<br>index=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': '5',
              'marker': {'color': '#19d3f3', 'pattern': {'shape': ''}},
              'name': '5',
              'offs